In [122]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
import csv
import urllib
import seaborn as sns
import matplotlib.pyplot as plt
import ssl
import pdfplumber
import urllib
import io
from io import BytesIO
from collections import Counter
import json
import os
import re
#from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.manifold import TSNE
from six.moves import cPickle
import gensim.models.word2vec as w2v
import tensorflow as tf
from tensorflow.python.framework.ops import reset_default_graph
# from tensorflow.python.framework import ops
# ops.reset_default_graph()
import multiprocessing

## import data

In [2]:
# this is just usernames
# https://www.kaggle.com/mrmorj/us-politicians-twitter-dataset
df = pd.read_csv('data/dataset.csv')

In [3]:
df.head()

,Name,Twitter_username,Account_start_time,Account_ID,Sex,Birthplace,Birthday,Age,Instagram_username,Political_party
0,A. Donald McEachin,RepMcEachin,2017-01-03T00:00:00Z,816181091673448448,male,Germany,1961-10-10T00:00:00Z,59.0,repmceachin,Democratic Party
1,Aaron Michlewitz,RepMichlewitz,2010-06-27T00:00:00Z,160246973,male,United States of America,1978-01-01T00:00:00Z,42.0,NaN,Democratic Party
2,Aaron Peskin,AaronPeskin,2010-11-13T00:00:00Z,215369273,male,United States of America,1964-06-17T00:00:00Z,56.0,apeskin52,Democratic Party
3,Aaron Peña,AaronPena,2007-10-31T00:00:00Z,9843332,male,United States of America,1959-06-08T00:00:00Z,61.0,NaN,Republican Party
4,Aaron Schock,aaronschock,2009-03-12T00:00:00Z,23951197,male,United States of America,1981-05-28T00:00:00Z,39.0,aaronschock,Republican Party


In [14]:
df.shape

(2514, 10)

In [4]:
# actual tweets
# https://www.reddit.com/r/datasets/comments/6fniik/over_one_million_tweets_collected_from_us/
# this data is 3 years old... 
# json_file = open('data/US_PoliticalTweets/tweets.json')
# data = json.load(json_file)
# tweets = json.load(json_file)

tweets = []
for line in open('data/US_PoliticalTweets/tweets.json', 'r'):
    tweets.append(json.loads(line))

In [5]:
tweets_df = pd.DataFrame.from_dict(tweets)

In [7]:
tweets_df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'favorite_count', 'favorited', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'retweet_count', 'retweeted', 'screen_name', 'source', 'text',
       'truncated', 'user_id', 'possibly_sensitive', 'extended_entities',
       'quoted_status_id', 'quoted_status_id_str', 'withheld_copyright',
       'withheld_in_countries', 'withheld_scope'],
      dtype='object')

In [10]:
# number of politicians
len(set(tweets_df.screen_name))

545

In [13]:
# average num of tweets
t_gp = tweets_df.groupby(['screen_name'])['id'].size().reset_index()
print('avg: {}'.format(t_gp.id.mean()))
print('median: {}'.format(t_gp.id.median()))

avg: 2281.412844036697
median: 2691.0


In [18]:
# most of these politicians (417 / 545) are in the other dataset which includes full names, party, age... 
len(set(tweets_df.screen_name).intersection(set(df.Twitter_username)))

417

## set up 

In [79]:
save_dir = '/Users/madeline.campbell/Documents/GitHub/analysis_projects/political_tweets/output/'

In [90]:
def try_load_or_process(filename, processor_fn, function_arg):
    load_fn = None
    save_fn = None
    if filename.endswith("json"):
        load_fn = load_json
        save_fn = save_json
    else:
        load_fn = load_bin
        save_fn = save_bin
    if os.path.exists(filename):
        return load_fn(filename)
    else:
        ret = processor_fn(function_arg)
        save_fn(ret, filename)
        return ret

def print_progress(current, maximum):
    sys.stdout.write("\r")
    sys.stdout.flush()
    sys.stdout.write(str(current) + "/" + str(maximum))
    sys.stdout.flush()

def save_bin(item, filename):
    with open(filename, "wb") as f:
        cPickle.dump(item, f)

def load_bin(filename):
    if os.path.exists(filename):
        with open(filename, "rb") as f:
            return cPickle.load(f)

def save_json(variable, filename):
    with io.open(filename, "w", encoding="utf-8") as f:
        f.write(json.dumps(variable, indent=4, ensure_ascii=False))

def load_json(filename):
    ret = None
    if os.path.exists(filename):
        try:
            with io.open(filename, "r", encoding="utf-8") as f:
                ret = json.load(f)
        except:
            pass
    return ret

In [40]:
# might need to stip text of names (@..), urls, hashtags
# try without stripping first? 
tweets_df.text

0          On a conference call about a weekend trip to I...
1                                  Being interviewed by KTHV
2          Being interviewed by KARN on his Arkansas Worl...
3                      On KWHN in Fort Smith, that's 1320 AM
4          Attending a Military of the Purple Heart Cerem...
                                 ...                        
1243365    Dismantling #DoddFrank returns us to the days ...
1243366    In the shadows of the #ComeyHearing, @HouseGOP...
1243367    @BetsyDeVosED How does a budget that cuts inve...
1243368    Thank you @POTUS @NikkiHaley for strong stance...
1243369    #WrongCHOICEAct will eliminate consumer protec...
Name: text, Length: 1243370, dtype: object

In [24]:
#tweets_df.text[0].split(' ')
tweets_df['text_split'] = tweets_df.text.str.split(' ')

In [152]:
# JSON file 
# 'data/stopwords.json'

en_stopwords = ["RT", "&amp;", "The", "w/", "We", "This", "...", "To", "&", "here:", "1", "As", "@", "2", "4", " ", "-", "In", "My", "I'm", "A", "In", "It", "It's", "On", "If", "'ll","'tis","'twas","'ve","10","39","a","a's","able","ableabout","about","above","abroad","abst","accordance","according","accordingly","across","act","actually","ad","added","adj","adopted","ae","af","affected","affecting","affects","after","afterwards","ag","again","against","ago","ah","ahead","ai","ain't","aint","al","all","allow","allows","almost","alone","along","alongside","already","also","although","always","am","amid","amidst","among","amongst","amoungst","amount","an","and","announce","another","any","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","ao","apart","apparently","appear","appreciate","appropriate","approximately","aq","ar","are","area","areas","aren","aren't","arent","arise","around","arpa","as","aside","ask","asked","asking","asks","associated","at","au","auth","available","aw","away","awfully","az","b","ba","back","backed","backing","backs","backward","backwards","bb","bd","be","became","because","become","becomes","becoming","been","before","beforehand","began","begin","beginning","beginnings","begins","behind","being","beings","believe","below","beside","besides","best","better","between","beyond","bf","bg","bh","bi","big","bill","billion","biol","bj","bm","bn","bo","both","bottom","br","brief","briefly","bs","bt","but","buy","bv","bw","by","bz","c","c'mon","c's","ca","call","came","can","can't","cannot","cant","caption","case","cases","cause","causes","cc","cd","certain","certainly","cf","cg","ch","changes","ci","ck","cl","clear","clearly","click","cm","cmon","cn","co","co.","com","come","comes","computer","con","concerning","consequently","consider","considering","contain","containing","contains","copy","corresponding","could","could've","couldn","couldn't","couldnt","course","cr","cry","cs","cu","currently","cv","cx","cy","cz","d","dare","daren't","darent","date","de","dear","definitely","describe","described","despite","detail","did","didn","didn't","didnt","differ","different","differently","directly","dj","dk","dm","do","does","doesn","doesn't","doesnt","doing","don","don't","done","dont","doubtful","down","downed","downing","downs","downwards","due","during","dz","e","each","early","ec","ed","edu","ee","effect","eg","eh","eight","eighty","either","eleven","else","elsewhere","empty","end","ended","ending","ends","enough","entirely","er","es","especially","et","et-al","etc","even","evenly","ever","evermore","every","everybody","everyone","everything","everywhere","ex","exactly","example","except","f","face","faces","fact","facts","fairly","far","farther","felt","few","fewer","ff","fi","fifteen","fifth","fifty","fify","fill","find","finds","fire","first","five","fix","fj","fk","fm","fo","followed","following","follows","for","forever","former","formerly","forth","forty","forward","found","four","fr","free","from","front","full","fully","further","furthered","furthering","furthermore","furthers","fx","g","ga","gave","gb","gd","ge","general","generally","get","gets","getting","gf","gg","gh","gi","give","given","gives","giving","gl","gm","gmt","gn","go","goes","going","gone","good","goods","got","gotten","gov","gp","gq","gr","great","greater","greatest","greetings","group","grouped","grouping","groups","gs","gt","gu","gw","gy","h","had","hadn't","hadnt","half","happens","hardly","has","hasn","hasn't","hasnt","have","haven","haven't","havent","having","he","he'd","he'll","he's","hed","hell","hello","help","hence","her","here","here's","hereafter","hereby","herein","heres","hereupon","hers","herself","herse”","hes","hi","hid","high","higher","highest","him","himself","himse”","his","hither","hk","hm","hn","home","homepage","hopefully","how","how'd","how'll","how's","howbeit","however","hr","ht","htm","html","http","hu","hundred","i","i'd","i'll","i'm","i've","i.e.","id","ie","if","ignored","ii","il","ill","im","immediate","immediately","importance","important","in","inasmuch","inc","inc.","indeed","index","indicate","indicated","indicates","information","inner","inside","insofar","instead","int","interest","interested","interesting","interests","into","invention","inward","io","iq","ir","is","isn","isn't","isnt","it","it'd","it'll","it's","itd","itll","its","itself","itse”","ive","j","je","jm","jo","join","jp","just","k","ke","keep","keeps","kept","keys","kg","kh","ki","kind","km","kn","knew","know","known","knows","kp","kr","kw","ky","kz","l","la","large","largely","last","lately","later","latest","latter","latterly","lb","lc","least","length","less","lest","let","let's","lets","li","like","liked","likely","likewise","line","little","lk","ll","long","longer","longest","look","looking","looks","low","lower","lr","ls","lt","ltd","lu","lv","ly","m","ma","made","mainly","make","makes","making","man","many","may","maybe","mayn't","maynt","mc","md","me","mean","means","meantime","meanwhile","member","members","men","merely","mg","mh","microsoft","might","might've","mightn't","mightnt","mil","mill","million","mine","minus","miss","mk","ml","mm","mn","mo","more","moreover","most","mostly","move","mp","mq","mr","mrs","ms","msie","mt","mu","much","mug","must","must've","mustn't","mustnt","mv","mw","mx","my","myself","myse”","mz","n","na","name","namely","nay","nc","nd","ne","near","nearly","necessarily","necessary","need","needed","needing","needn't","neednt","needs","neither","net","netscape","never","neverf","neverless","nevertheless","new","newer","newest","next","nf","ng","ni","nine","ninety","nl","no","no-one","nobody","non","none","nonetheless","noone","nor","normally","nos","not","noted","nothing","notwithstanding","novel","now","nowhere","np","nr","nu","null","number","numbers","nz","o","obtain","obtained","obviously","of","off","often","oh","ok","okay","old","older","oldest","om","omitted","on","once","one","one's","ones","only","onto","open","opened","opening","opens","opposite","or","ord","order","ordered","ordering","orders","org","other","others","otherwise","ought","oughtn't","oughtnt","our","ours","ourselves","out","outside","over","overall","owing","own","p","pa","page","pages","part","parted","particular","particularly","parting","parts","past","pe","per","perhaps","pf","pg","ph","pk","pl","place","placed","places","please","plus","pm","pmid","pn","point","pointed","pointing","points","poorly","possible","possibly","potentially","pp","pr","predominantly","present","presented","presenting","presents","presumably","previously","primarily","probably","problem","problems","promptly","proud","provided","provides","pt","put","puts","pw","py","q","qa","que","quickly","quite","qv","r","ran","rather","rd","re","readily","really","reasonably","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","reserved","respectively","resulted","resulting","results","right","ring","ro","room","rooms","round","ru","run","rw","s","sa","said","same","saw","say","saying","says","sb","sc","sd","se","sec","second","secondly","seconds","section","see","seeing","seem","seemed","seeming","seems","seen","sees","self","selves","sensible","sent","serious","seriously","seven","seventy","several","sg","sh","shall","shan't","shant","she","she'd","she'll","she's","shed","shell","shes","should","should've","shouldn","shouldn't","shouldnt","show","showed","showing","shown","showns","shows","si","side","sides","significant","significantly","similar","similarly","since","sincere","site","six","sixty","sj","sk","sl","slightly","sm","small","smaller","smallest","sn","so","some","somebody","someday","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","sr","st","state","states","still","stop","strongly","su","sub","substantially","successfully","such","sufficiently","suggest","sup","sure","sv","sy","system","sz","t","t's","take","taken","taking","tc","td","tell","ten","tends","test","text","tf","tg","th","than","thank","thanks","thanx","that","that'll","that's","that've","thatll","thats","thatve","the","their","theirs","them","themselves","then","thence","there","there'd","there'll","there're","there's","there've","thereafter","thereby","thered","therefore","therein","therell","thereof","therere","theres","thereto","thereupon","thereve","these","they","they'd","they'll","they're","they've","theyd","theyll","theyre","theyve","thick","thin","thing","things","think","thinks","third","thirty","this","thorough","thoroughly","those","thou","though","thoughh","thought","thoughts","thousand","three","throug","through","throughout","thru","thus","til","till","tip","tis","tj","tk","tm","tn","to","today","together","too","took","top","toward","towards","tp","tr","tried","tries","trillion","truly","try","trying","ts","tt","turn","turned","turning","turns","tv","tw","twas","twelve","twenty","twice","two","tz","u","ua","ug","uk","um","un","under","underneath","undoing","unfortunately","unless","unlike","unlikely","until","unto","up","upon","ups","upwards","us","use","used","useful","usefully","usefulness","uses","using","usually","uucp","uy","uz","v","va","value","various","vc","ve","versus","very","vg","vi","via","viz","vn","vol","vols","vs","vu","w","want","wanted","wanting","wants","was","wasn","wasn't","wasnt","way","ways","we","we'd","we'll","we're","we've","web","webpage","website","wed","welcome","well","wells","went","were","weren","weren't","werent","weve","wf","what","what'd","what'll","what's","what've","whatever","whatll","whats","whatve","when","when'd","when'll","when's","whence","whenever","where","where'd","where'll","where's","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","which","whichever","while","whilst","whim","whither","who","who'd","who'll","who's","whod","whoever","whole","wholl","whom","whomever","whos","whose","why","why'd","why'll","why's","widely","width","will","willing","wish","with","within","without","won","won't","wonder","wont","words","work","worked","working","works","world","would","would've","wouldn","wouldn't","wouldnt","ws","www","x","y","ye","year","years","yes","yet","you","you'd","you'll","you're","you've","youd","youll","young","younger","youngest","your","youre","yours","yourself","yourselves","youve","yt","yu","z","za","zero","zm","zr"]
# remove stopwords
# this list feels a little long but OK 

tweets_df['text_gowords'] = [' '.join([y for y in x if y not in en_stopwords]) for x in tweets_df['text_split']]

In [153]:
tweets_df['textgo_split'] = tweets_df.text_gowords.str.split(' ')

In [154]:
tweets_df.textgo_split[0]

['conference', 'weekend', 'trip', 'Iraq', 'visit', 'Arkansas', 'troops']

In [155]:
def get_word_frequencies(corpus):
    frequencies = Counter()
    for sentence in corpus:
        for word in sentence:
            frequencies[word] += 1
    freq = frequencies.most_common()
    return freq

In [156]:
print("Getting word frequencies")
filename = os.path.join(save_dir, "frequencies.json")
frequencies =  get_word_frequencies(tweets_df.textgo_split)
vocab_size = len(frequencies)
print("Unique words: " + str(vocab_size))

# word_freq = get_word_frequencies()

Getting word frequencies
Unique words: 1682639


In [157]:
word_freq

[('RT', 266209),
 ('&amp;', 216536),
 ('I', 127745),
 ('', 89389),
 ('The', 61207),
 ('We', 47356),
 ('-', 45323),
 ('w/', 43713),
 ('House', 40591),
 ('Great', 35350),
 ('Thanks', 33463),
 ('Thank', 32517),
 ('My', 29721),
 ('support', 29717),
 ('time', 27319),
 ('This', 26137),
 ('health', 23494),
 ('American', 23222),
 ('Congress', 23217),
 ('here:', 22308),
 ('people', 21197),
 ('Happy', 20849),
 ('Today', 20548),
 ('Senate', 19701),
 ('Americans', 19693),
 ('discuss', 19640),
 ("I'm", 19495),
 ('Act', 18398),
 ('2', 18162),
 ('President', 18156),
 ('Proud', 17554),
 ('today.', 17529),
 ('A', 17163),
 ('meeting', 16321),
 ('care', 16272),
 ('students', 16156),
 ('hearing', 15995),
 ('vote', 15979),
 ('U.S.', 15920),
 ('honor', 15613),
 ('In', 15499),
 ('US', 15424),
 ('It', 15293),
 ('day', 15252),
 ('Congrats', 15169),
 ('Watch', 15028),
 ('families', 14939),
 ('protect', 14834),
 ('jobs', 14660),
 ('office', 14481),
 ("It's", 14403),
 ('women', 14353),
 ('On', 14298),
 ('live', 1

In [75]:
def get_word2vec(sentences):
    num_workers = multiprocessing.cpu_count()
    num_features = 200
    epoch_count = 10
    sentence_count = len(sentences)
    w2v_file = os.path.join(save_dir, "word_vectors.w2v")
    word2vec = None
    if os.path.exists(w2v_file):
        print("w2v model loaded from " + w2v_file)
        word2vec = w2v.Word2Vec.load(w2v_file)
    else:
        word2vec = w2v.Word2Vec(sg=1,
                                seed=1,
                                workers=num_workers,
                                size=num_features,
                                min_count=min_frequency_val,
                                window=5,
                                sample=0)

        print("Building vocab...")
        word2vec.build_vocab(sentences)
        print("Word2Vec vocabulary length:", len(word2vec.wv.vocab))
        print("Training...")
        word2vec.train(sentences, total_examples=sentence_count, epochs=epoch_count)
        print("Saving model...")
        word2vec.save(w2v_file)
    return word2vec

In [126]:
def create_embeddings(word2vec):
    all_word_vectors_matrix = word2vec.wv.syn0
    num_words = len(all_word_vectors_matrix)
    vocab = word2vec.wv.vocab.keys()
    vocab_len = len(vocab)
    dim = len(word2vec.wv[vocab]) #word2vec.wv[vocab[0]].shape[0]
    dim = 200
    embedding = np.empty((num_words, dim), dtype=np.float32)
    metadata = ""
    for i, word in enumerate(vocab):
        embedding[i] = word2vec.wv[word]
        metadata += word + "\n"
    metadata_file = os.path.join(save_dir, "metadata.tsv")
    with io.open(metadata_file, "w", encoding="utf-8") as f:
        f.write(metadata)

    reset_default_graph() #tf.reset_default_graph()
    sess = tf.compat.v1.InteractiveSession()
    X = tf.Variable([0.0], name='embedding')
    tf.compat.v1.disable_eager_execution()
    place = tf.compat.v1.placeholder(tf.float32, shape=embedding.shape)
    set_x = tf.compat.v1.assign(X, place, validate_shape=False)
    sess.run(tf.compat.v1.global_variables_initializer())
    sess.run(set_x, feed_dict={place: embedding})

    summary_writer = tf.summary.FileWriter(save_dir, sess.graph)
#     config = projector.ProjectorConfig()
#     embedding_conf = config.embeddings.add()
#     embedding_conf.tensor_name = 'embedding:0'
#     embedding_conf.metadata_path = 'metadata.tsv'
#     projector.visualize_embeddings(summary_writer, config)

    save_file = os.path.join(save_dir, "model.ckpt")
    print("Saving session...")
    saver = tf.train.Saver()
    saver.save(sess, save_file)

In [98]:
print("Instantiating word2vec model")
min_frequency_val = 6
word2vec = get_word2vec(tweets_df.textgo_split)

Building vocab...
Word2Vec vocabulary length: 122001
Training...
Saving model...


In [112]:
vocab = word2vec.wv.vocab.keys()
vocab_len = len(vocab)
print("word2vec vocab contains " + str(vocab_len) + " items.")
dim0 = len(word2vec.wv[vocab]) # word2vec.wv[vocab[0]].shape[0]
print("word2vec items have " + str(dim0) + " features.")

word2vec vocab contains 122001 items.
word2vec items have 122001 features.


In [149]:
all_word_vectors_matrix = word2vec.wv.syn0
num_words = len(all_word_vectors_matrix)
vocab = word2vec.wv.vocab.keys()
vocab_len = len(vocab)
dim = len(word2vec.wv[vocab]) #word2vec.wv[vocab[0]].shape[0]
dim = 200
embedding = np.empty((num_words, dim), dtype=np.float32)
metadata = ""
for i, word in enumerate(vocab):
    embedding[i] = word2vec.wv[word]
    metadata += word + "\n"
metadata_file = os.path.join(save_dir, "metadata.tsv")
with io.open(metadata_file, "w", encoding="utf-8") as f:
    f.write(metadata)

reset_default_graph() #tf.reset_default_graph()
sess = tf.compat.v1.InteractiveSession()
X = tf.Variable([0.0], name='embedding')
tf.compat.v1.disable_eager_execution()
place = tf.compat.v1.placeholder(tf.float32, shape=embedding.shape)
set_x = tf.compat.v1.assign(X, place, validate_shape=False)
# sess.run(tf.compat.v1.global_variables_initializer())
# sess.run(set_x, feed_dict={place: embedding})

# summary_writer = tf.summary.FileWriter(save_dir, sess.graph)

/Users/madeline.campbell/.pyenv/versions/3.6.8/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


ValueError: Shapes (1,) and (122001, 200) are incompatible

In [150]:
 #[vocab[0]].shape[0]
#len(word2vec.wv[vocab])
#len(embedding[i])
place

<tf.Tensor 'Placeholder:0' shape=(122001, 200) dtype=float32>

In [127]:
print("Creating tensorboard embeddings")
create_embeddings(word2vec)

Creating tensorboard embeddings


/Users/madeline.campbell/.pyenv/versions/3.6.8/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


ValueError: Shapes (1,) and (122001, 200) are incompatible

In [94]:
def most_similar(input_word, num_similar):
    sim = word2vec.wv.most_similar(input_word, topn=num_similar)
    output = []
    found = []
    for item in sim:
        w, n = item
        found.append(w)
    output = [input_word, found]
    return output

In [95]:
def t_sne_scatterplot(word):
    vocab = word2vec.wv.vocab.keys()
    vocab_len = len(vocab)
    dim0 = word2vec.wv[vocab[0]].shape[0]
    arr = np.empty((0, dim0), dtype='f')
    w_labels = [word]
    nearby = word2vec.wv.similar_by_word(word, topn=num_similar)
    arr = np.append(arr, np.array([word2vec[word]]), axis=0)
    for n in nearby:
        w_vec = word2vec[n[0]]
        w_labels.append(n[0])
        arr = np.append(arr, np.array([w_vec]), axis=0)

    tsne = TSNE(n_components=2, random_state=1)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]

    plt.rc("font", size=16)
    plt.figure(figsize=(16, 12), dpi=80)
    plt.scatter(x_coords[0], y_coords[0], s=800, marker="o", color="blue")
    plt.scatter(x_coords[1:], y_coords[1:], s=200, marker="o", color="red")

    for label, x, y in zip(w_labels, x_coords, y_coords):
        plt.annotate(label.upper(), xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()-50, x_coords.max()+50)
    plt.ylim(y_coords.min()-50, y_coords.max()+50)
    filename = os.path.join(plot_dir, word + "_tsne.png")
    plt.savefig(filename)
    plt.close()

In [96]:
def calculate_t_sne():
    vocab = word2vec.wv.vocab.keys()
    vocab_len = len(vocab)
    arr = np.empty((0, dim0), dtype='f')
    labels = []
    vectors_file = os.path.join(save_dir, "vocab_vectors.npy")
    labels_file = os.path.join(save_dir, "labels.json")
    if os.path.exists(vectors_file) and os.path.exists(labels_file):
        print("Loading pre-saved vectors from disk")
        arr = load_bin(vectors_file)
        labels = load_json(labels_file)
    else:
        print("Creating an array of vectors for each word in the vocab")
        for count, word in enumerate(vocab):
            if count % 50 == 0:
                print_progress(count, vocab_len)
            w_vec = word2vec[word]
            labels.append(word)
            arr = np.append(arr, np.array([w_vec]), axis=0)
        save_bin(arr, vectors_file)
        save_json(labels, labels_file)

    x_coords = None
    y_coords = None
    x_c_filename = os.path.join(save_dir, "x_coords.npy")
    y_c_filename = os.path.join(save_dir, "y_coords.npy")
    if os.path.exists(x_c_filename) and os.path.exists(y_c_filename):
        print("Reading pre-calculated coords from disk")
        x_coords = load_bin(x_c_filename)
        y_coords = load_bin(y_c_filename)
    else:
        print("Computing T-SNE for array of length: " + str(len(arr)))
        tsne = TSNE(n_components=2, random_state=1, verbose=1)
        np.set_printoptions(suppress=True)
        Y = tsne.fit_transform(arr)
        x_coords = Y[:, 0]
        y_coords = Y[:, 1]
        print("Saving coords.")
        save_bin(x_coords, x_c_filename)
        save_bin(y_coords, y_c_filename)
    return x_coords, y_coords, labels, arr

In [ ]:
print("Calculating T-SNE for word2vec model")
x_coords, y_coords, labels, arr = calculate_t_sne()
